In [1]:
import re
import numpy as np
import tensorflow as tf
import os
import glob
from sklearn.model_selection import train_test_split

### 1. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("Examples:\n", raw_corpus[:10])

Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face [Verse 1]', "Somethin' ain't right when we talkin'"]


#### <데이터 필터링 구상> 

1. 사람, 영역 이름 빼기 '[Hook]', [Verse 1], [Pre-Hook]
2. 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외
3. 문장부호 
4. 대소문자
5. 특수문자

### 2. 데이터 정제

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if sentence[-1] == "]": continue  # 문장의 끝이 ] 인 문장은 건너뜁니다.
    if idx > 9: break   # 일단 문장 10개만 확인
        
    print(sentence)

I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Somethin' ain't right when we talkin'


In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue # 이 행 빼면 왜 index out of range 나오는지 잘 모르겠음
    if sentence[-1] == "]": continue
    if len(sentence.split()) > 13 : continue
    corpus.append(preprocess_sentence(sentence))
    
corpus[:50]

['<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> look like you hidin your problems <end>',
 '<start> really you never was solid <end>',
 '<start> no , you can t son me <end>',
 '<start> you won t never get to run me <end>',
 '<start> just when shit look out of reach <end>',
 '<start> i reach back like one , three <end>',
 '<start> that s when they smile in my face <end>',
 '<start> whole time they wanna take my place <end>',
 '<start> whole time they wanna take my place <end>',
 '<start> whole time they wanna take my place <end>',
 '<sta

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters='',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   5  91 ...   0   0   0]
 [  2  42 133 ...   0   0   0]
 [  2   5  39 ...   0   0   0]
 ...
 [  2   6 456 ...   0   0   0]
 [  2   8  38 ...   0   0   0]
 [  2   5  93 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f3c1c1c5e50>


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :14]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:15]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2   5  91 103  59  31 166   4  11 133  24  29  10  12]
[  5  91 103  59  31 166   4  11 133  24  29  10  12   3]


### 3. 평가 데이터셋 분리

In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                    tgt_input,
                                                    test_size=0.2,
                                                    random_state=7)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (129670, 14)
Target Train: (129670, 14)


In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 512
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((512, 14), (512, 14)), types: (tf.int32, tf.int32)>

### 3. 학습시키기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(512, 14, 7001), dtype=float32, numpy=
array([[[-5.63415306e-05, -9.41520266e-05, -2.51911668e-04, ...,
          1.96514142e-04, -4.63830482e-04,  1.46236940e-04],
        [ 6.22594671e-05, -1.80944684e-04, -3.85232503e-04, ...,
          1.41872311e-04, -4.14292532e-04,  1.89357423e-04],
        [ 6.64875406e-05, -1.28448417e-04, -4.49140294e-04, ...,
          2.79058760e-04, -6.47703360e-04,  3.29607465e-05],
        ...,
        [-6.15862606e-04,  5.11600170e-04,  6.17681595e-04, ...,
         -6.04003202e-04,  4.64261509e-04, -4.05132829e-04],
        [-9.36098106e-04,  2.75941828e-04,  5.10788290e-04, ...,
         -6.17448823e-04,  2.50672281e-04,  9.55955693e-05],
        [-4.24149737e-04,  6.37487465e-05,  6.04859670e-04, ...,
         -2.62483838e-04,  1.62662225e-04,  5.51580510e-04]],

       [[-5.63415306e-05, -9.41520266e-05, -2.51911668e-04, ...,
          1.96514142e-04, -4.63830482e-04,  1.46236940e-04],
        [-1.18285338e-04,  1.21915786e-04, -3.

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
253/253 [==============================] - 48s 190ms/step - loss: 3.6679
Epoch 2/10
253/253 [==============================] - 49s 193ms/step - loss: 3.0840
Epoch 3/10
253/253 [==============================] - 49s 193ms/step - loss: 2.8959
Epoch 4/10
253/253 [==============================] - 49s 194ms/step - loss: 2.7619
Epoch 5/10
253/253 [==============================] - 49s 194ms/step - loss: 2.6514
Epoch 6/10
253/253 [==============================] - 49s 194ms/step - loss: 2.5509
Epoch 7/10
253/253 [==============================] - 49s 194ms/step - loss: 2.4573
Epoch 8/10
253/253 [==============================] - 50s 197ms/step - loss: 2.3676
Epoch 9/10
253/253 [==============================] - 49s 195ms/step - loss: 2.2806
Epoch 10/10
253/253 [==============================] - 49s 194ms/step - loss: 2.1948


In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '